In [2]:
import rtde_control
import rtde_receive
from rtde_control import Path, PathEntry
import rtde_io
from robotiq_gripper_control import RobotiqGripper
import rotation_matrix as rm
import time
import numpy as np
import rotation_matrix as rm
import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'rtde_control'

In [13]:
def connect_robot(ip = "192.168.2.1"):
    rtde_c = rtde_control.RTDEControlInterface(ip) #IP address found on robot
    rtde_r = rtde_receive.RTDEReceiveInterface(ip)
    rtde_io_set = rtde_io.RTDEIOInterface(ip)
    return rtde_c, rtde_r, rtde_io_set

rtde_c, rtde_r, rtde_io_set = connect_robot()

In [ ]:
print("Activating Gripper")
gripper = RobotiqGripper(rtde_c)
gripper.activate()  # returns to previous position after activation
gripper.set_force(10)  # from 0 to 100 %
gripper.set_speed(10)  # from 0 to 100 %
print("Gripper activated")

In [53]:
# open csv file
test_flasks = pd.read_csv('../pouring_simulation/output/summary_flask_test_50.csv')

In [54]:
#print first 5 rows
test_flasks.head()

,scene_number,path,rotationSpeed,stop_angle,pause_time,volume_start,volume_poured,volume_received,spilled_volume,real_poured_volume,real_received_volume,real_spilled_volume
0,1102,../../output/CellFlask/Flask_75_1799_4,0.03,4.0,1.8,72.3750,0.122500,0.010000,0.1125,0.0,0.0,0.0
1,2588,../../output/CellFlask/Flask_135_1399_26,0.03,26.0,1.4,130.5650,115.852501,107.027496,8.8250,0.0,0.0,0.0
2,1159,../../output/CellFlask/Flask_80_600_18,0.03,18.0,0.6,77.2650,38.947498,38.767502,0.1800,0.0,0.0,0.0
3,2827,../../output/CellFlask/Flask_145_1399_4,0.03,4.0,1.4,140.1000,0.402500,0.220000,0.1825,0.0,0.0,0.0
4,231,../../output/CellFlask/Flask_20_1799_12,0.03,12.0,1.8,19.5725,0.575000,0.510000,0.0650,0.0,0.0,0.0


## Manually move robot to start position and read the position

In [ ]:
start_pos = rtde_r.getActualTCPPose()

In [ ]:
print(start_pos)

# Start here:

## Input sample number:

In [43]:
sample_number = 0

### Loading TCP file for movement

In [62]:
scene_path = test_flasks.iloc[sample_number]['path']
# delete first five characters of the path
scene_path = scene_path[5:]
scene_path = "../pouring_simulation/" + scene_path + "/TCP.txt"
data_points = np.loadtxt(scene_path, delimiter=',', skiprows=1)

# convert from inches to meters
data_points[:, 0] = data_points[:, 0] * 0.0254
data_points[:, 1] = data_points[:, 1] * 0.0254

# create list of positions
positions = []
for i in range(data_points.shape[0]):
    positions.append([-data_points[i,1], -data_points[i,0], 0.0, 0.0, 0.0, data_points[i,2]]) # will move around x, y of tool and rotate around z of tool --> to be updated for different setups

positions_converted = []
for i in range(data_points.shape[0]):
    # if none of the data entries is 0
    if not (positions[i][0] == 0 or positions[i][1] == 0 or positions[i][5] == 0):
        positions_converted.append(rm.PoseTrans(start_pos, positions[i])) # transform from tool coordinate system to base coordinate system
        

### Calculate speed


In [ ]:
# get first 5 x and z positions 
positions = positions_converted[15:30]

# only get x, y, z positions
positions = [x[0:3] for x in positions]


#positions = [x[0:3:2] for x in positions]

# calculate distances between positions in meters
distances = []
for i in range(len(positions)-1):
    distances.append(np.linalg.norm(np.subtract(positions[i+1], positions[i])))

print(distances)

speed = []

# calculate speed for a frequency of 60 Hz in m/s
for i in range(len(distances)):
    speed.append(distances[i]*89)
    #speed.append(distances[i]*60)

print(speed)

# calculate average speed
avg_speed = np.mean(speed)
print("Speed in m/s: ", avg_speed)

### Calculate pause time and find pause point

In [ ]:
# get position of the first duplicate converted_position that is not position 0
for i in range(len(positions_converted)):
    if positions_converted[i] == positions_converted[0]:
        continue
    elif positions_converted[i] == positions_converted[i-1] == positions_converted[i-2] == positions_converted[i-3] == positions_converted[i-4]:
        print(i)
        break

# count the values that are the same as i
count = 0
for j in range(i, len(positions_converted)):
    if positions_converted[j] == positions_converted[i]:
        count += 1
    else:
        break
print(count)

# split positions_converted into two lists
positions_converted1 = positions_converted[0:i]
positions_converted2 = positions_converted[i:]

### Show pouring settings from simulation

In [65]:
print("Start Volume: ", test_flasks.iloc[sample_number]['volume_start'], "mL")
print("Predicted Poured Volume: ", test_flasks.iloc[sample_number]['volume_poured'], "mL")
print("Predicted Remaining Volume: ", float(test_flasks.iloc[sample_number]['volume_start'])-float(test_flasks.iloc[sample_number]['volume_poured']), "mL")
print("Predicted Received Volume: ", test_flasks.iloc[sample_number]['volume_received'], "mL")
print("Predicted Spilled Volume: ", test_flasks.iloc[sample_number]['spilled_volume'], "mL")
print("\n")
print("Stop Angle: ", test_flasks.iloc[sample_number]['stop_angle'], "degrees")
print("Stop Time: ", test_flasks.iloc[sample_number]['pause_time'], "seconds")

Start Volume:  72.375 mL
Predicted Poured Volume:  0.1225 mL
Predicted Remaining Volume:  72.2525 mL
Predicted Received Volume:  0.01 mL
Predicted Spilled Volume:  0.1125 mL


Stop Angle:  4.0 degrees
Stop Time:  1.8 seconds


### Fill Flask

In [ ]:
rtde_c.moveL(start_pos, 0.5, 0.5)
gripper.open()

### Place Flask in gripper

In [ ]:
gripper.close()

### Do pouring

In [39]:
velocity = avg_speed #0.5
acceleration = 1.5
blend_1 = 0.0
blend_i = 0.001
blend_3 = 0.0
#path_pose1 = [start_pos_flask2[0], start_pos_flask2[1], start_pos_flask2[2], start_pos_flask2[3], start_pos_flask2[4], start_pos_flask2[5], velocity, acceleration, blend_1]
path = []
#path.append(path_pose1)
for i in range(len(positions_converted1)-1):
    path.append([positions_converted1[i][0], positions_converted1[i][1], positions_converted1[i][2], positions_converted1[i][3], positions_converted1[i][4], positions_converted1[i][5], velocity, acceleration, blend_i])

path.append([positions_converted1[-1][0], positions_converted1[-1][1], positions_converted1[-1][2], positions_converted1[-1][3], positions_converted1[-1][4], positions_converted1[-1][5], velocity, acceleration, 0])
rtde_c.moveL(path)

time.sleep(count/89)
#time.sleep(count/60)

path_2 = []
for i in range(len(positions_converted2)-1):
    path_2.append([positions_converted2[i][0], positions_converted2[i][1], positions_converted2[i][2], positions_converted2[i][3], positions_converted2[i][4], positions_converted2[i][5], velocity, acceleration, blend_i])

path_2.append([positions_converted2[-1][0], positions_converted2[-1][1], positions_converted2[-1][2], positions_converted2[-1][3], positions_converted2[-1][4], positions_converted2[-1][5], velocity, acceleration, blend_3])

rtde_c.moveL(path_2)

rtde_c.stopScript()

True

In [ ]:
gripper.open()

### Measure results, write to excel file and restart